# VacationPy
----

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_data = pd.read_csv("../output_data/cities.csv")
weather_data

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Mookane,3,BW,1619917384,72,-23.6880,26.6595,56.61,12.33
1,Hong Kong,90,HK,1619917254,87,22.2855,114.1577,80.01,3.04
2,East London,0,ZA,1619917384,67,-33.0153,27.9116,60.12,5.91
3,Carnarvon,100,AU,1619917385,64,-24.8667,113.6333,68.00,8.05
4,Kapaa,40,US,1619917385,69,22.0752,-159.3190,80.60,13.80
...,...,...,...,...,...,...,...,...,...
539,Sandakan,20,MY,1619917430,83,5.8402,118.1179,82.40,3.44
540,Santa Vitória,100,BR,1619917535,74,-18.8386,-50.1214,69.21,0.81
541,Labuan,20,MY,1619917535,83,5.2767,115.2417,82.40,4.61
542,Lakhdenpokh'ya,24,RU,1619917536,51,61.5198,30.1992,34.00,4.59


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]].astype(float)

humidity = weather_data["Humidity"].astype(float)

In [5]:
# Plot Heatmap
fig = gmaps.figure(center=(15.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
max_intensity

100.0

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
# Narrow the cities with wind speed less yhaan 15 mph, cloudiness less than/equal to 10 and temperature range of 70-85

narrowed_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 6) & (weather_data["Cloudiness"] <= 5) & 
                                    (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 85)].dropna()

narrowed_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
13,Sitrah,0,BH,1619917387,65,26.1547,50.6206,80.60,4.61
30,Salalah,0,OM,1619917392,83,17.0151,54.0924,82.40,3.44
50,Nuevo México,0,MX,1619917397,19,20.7500,-103.4333,82.99,5.99
142,Tura,5,IN,1619917423,63,25.5198,90.2201,76.82,5.68
154,Pozo Colorado,0,PY,1619917426,53,-23.4923,-58.7972,70.93,5.97
197,Faanui,2,PF,1619917439,67,-16.4833,-151.7500,80.78,3.83
220,Porto Walter,5,BR,1619917445,81,-8.2686,-72.7439,73.27,1.72
224,Napa,0,US,1619917446,47,38.4832,-122.3344,81.00,3.00
228,Dicabisagan,2,PH,1619917447,65,17.0818,122.4157,84.54,3.87
236,Baghdad,0,IQ,1619917302,33,33.3406,44.4009,71.60,1.14


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
# Create a hotel DataFrame
hotel_df = narrowed_city_df.loc[:,["City", "Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
13,Sitrah,BH,26.1547,50.6206,
30,Salalah,OM,17.0151,54.0924,
50,Nuevo México,MX,20.7500,-103.4333,
142,Tura,IN,25.5198,90.2201,
154,Pozo Colorado,PY,-23.4923,-58.7972,
197,Faanui,PF,-16.4833,-151.7500,
220,Porto Walter,BR,-8.2686,-72.7439,
224,Napa,US,38.4832,-122.3344,
228,Dicabisagan,PH,17.0818,122.4157,
236,Baghdad,IQ,33.3406,44.4009,


In [13]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type": "hotel",
             "keyword": "hotel",
             "radius": 5000,
             "key": g_key}

In [14]:
for index, row in hotel_df.iterrows():
    # Get city name, lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # Add keyword to params dict
    params["location"] = f"{lat},{lng}"
    
    # Assemble url and make API request
    print(f"Retrieving results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # Extract results
    results = response["results"]
    
    # Save the hotel name to DataFrame
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        
    # If no hotel is available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("--------------------------")
    
    # Wait 1 second to make another API request to avoid SSL Error
    time.sleep(1)
    
# Print end of search after complete
print("-------End of Search------")

Retrieving results for Index 13: Sitrah.
Closest hotel in Sitrah is Al Bander Resort - منتجع البندر.
--------------------------
Retrieving results for Index 30: Salalah.
Closest hotel in Salalah is HAMDAN PLAZA HOTEL SALALAH.
--------------------------
Retrieving results for Index 50: Nuevo México.
Closest hotel in Nuevo México is Hyatt Regency Andares Guadalajara.
--------------------------
Retrieving results for Index 142: Tura.
Closest hotel in Tura is Hotel Polo Orchid.
--------------------------
Retrieving results for Index 154: Pozo Colorado.
Closest hotel in Pozo Colorado is Hotel Pozo Colorado.
--------------------------
Retrieving results for Index 197: Faanui.
Closest hotel in Faanui is InterContinental Bora Bora Resort Thalasso Spa.
--------------------------
Retrieving results for Index 220: Porto Walter.
Closest hotel in Porto Walter is Hotel das Irmãs..
--------------------------
Retrieving results for Index 224: Napa.
Missing field/result... skipping.
-------------------

In [15]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
13,Sitrah,BH,26.1547,50.6206,Al Bander Resort - منتجع البندر
30,Salalah,OM,17.0151,54.0924,HAMDAN PLAZA HOTEL SALALAH
50,Nuevo México,MX,20.7500,-103.4333,Hyatt Regency Andares Guadalajara
142,Tura,IN,25.5198,90.2201,Hotel Polo Orchid
154,Pozo Colorado,PY,-23.4923,-58.7972,Hotel Pozo Colorado
197,Faanui,PF,-16.4833,-151.7500,InterContinental Bora Bora Resort Thalasso Spa
220,Porto Walter,BR,-8.2686,-72.7439,Hotel das Irmãs.
224,Napa,US,38.4832,-122.3344,
228,Dicabisagan,PH,17.0818,122.4157,Palanan River
236,Baghdad,IQ,33.3406,44.4009,Baghdad Hotel


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the hotel layer to the map
fig.add_layer(markers)

# Display figure
fig



Figure(layout=FigureLayout(height='420px'))